In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

CH4_KG_PER_KCF = 19.17

In [2]:
# Set matplotlib font:
plt.rcParams['font.family'] = 'Helvetica Neue'

In [ ]:
def terraform_cost(usd_per_kg_co2: float, usd_per_kg_h2: float) -> float:
  """Returns the cost of Terraform methane in $/kg."""
  kg_co2_per_kg_ch4 = 2.74
  kg_h2_per_kg_ch4 = 0.251
  cost_scale_factor = 1.195

  co2_cost = usd_per_kg_co2 * kg_co2_per_kg_ch4
  h2_cost = usd_per_kg_h2 * kg_h2_per_kg_ch4
  total_cost = (co2_cost + h2_cost) * cost_scale_factor

  return total_cost

res1 = terraform_cost(78/1000, 0.89)
print(res1)

In [ ]:
x_axis_values = np.linspace(78.0, 250, 5)
y_axis_values = np.linspace(0.89, 2.5, 5)

def graph(x_axis_values, y_axis_values, units: str):
  if units not in ["kg", "kcf"]:
    raise ValueError("units must be 'kg' or 'kcf'")

  X, Y = np.meshgrid(x_axis_values, y_axis_values)

  Z = np.zeros_like(X)
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      co2_price_per_kg = X[i, j] / 1000
      h2_price_per_kg = Y[i, j]
      cost_per_kg = terraform_cost(co2_price_per_kg, h2_price_per_kg)

      if units == "kcf":
        cost_per_kcf = cost_per_kg * CH4_KG_PER_KCF
        Z[i, j] = cost_per_kcf
      else:
        Z[i, j] = cost_per_kg

  fig, ax = plt.subplots()
  c = ax.pcolormesh(X, Y, Z, cmap='YlOrRd')

  # fig.colorbar(c, ax=ax)
  plt.xlabel('$/kg CO2 (Direct Air Capture)')
  plt.ylabel('$/kg H2 (Electrolysis)')

  # Make the title bold:
  plt.title('Cost of Terraform Industries Green Methane', fontweight='bold')

  plt.xticks(x_axis_values)
  plt.yticks(y_axis_values)

  # Format the x and y labels to include a $ sign:
  ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: f'${x:.0f}'))
  ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y, loc: f'${y:.2f}'))

  # Add labels to each sqaure:
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      plt.text(X[i, j], Y[i, j], f'${Z[i, j]:.2f}', color='black', ha='center', va='center')

  plt.tight_layout()

  # Save the plot:
  plt.savefig(f'terraform_cost_{units}.png', dpi=300)

  plt.show()

graph(x_axis_values, y_axis_values, "kg")
graph(x_axis_values, y_axis_values, "kcf")